# Agenda: Dates and times in Pandas!

1. Dates and times in programming
2. Parsing dates with Pandas
3. The `.dt` accessor
4. Loading multiple files into a single data frame
5. How can you parse odd date formats?
6. Grouping with dates
7. Time deltas
8. Time series
9. Resampling

# Dates and times in programming

The most natural (and reasonable) way to to think about dates and times is as specific, unique moments in time. I can identify a "datetime" at various points in my life, and each of these is unique:

- When I was born
- When I graduated from university
- When this meeting started
- When this meeting ends

In programming, we describe these as "datetime" or "timestamp" objects.

We also talk about spans of time. Here, there is no hour/minute/second or year/month/day. Rather, it's "10 minutes long" or "3 days long."  Some examples:

- This meeting will last for 1 hour
- My life, so far, has been about 51.5 years.
- The pandemic has been going on for about 2 years now

This kind of measurement is known as a "time delta," or an "interval."

You can do some basic date+time math:

- timestamp + interval = timestamp
- timestamp - timestamp = interval

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

filename = '../data/nyc_taxi_2019-01.csv'
df = pd.read_csv(filename, 
                 usecols=['tpep_pickup_datetime',
                          'tpep_dropoff_datetime',
                          'passenger_count', 'total_amount', 'trip_distance'])


In [2]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,total_amount
0,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,9.95
1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,16.30
2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,5.80
3,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,7.55
4,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,55.55


In [3]:
df.dtypes

tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count            int64
trip_distance            float64
total_amount             float64
dtype: object

In [4]:
# how can I retrieve parts of the dates and times for dropoff and pickup?
# the answer is: we'll need to tell Pandas to parse those columns as dates


In [5]:
# if I want Pandas to parse one or more columns as datetimes, then
# I can pass those column names in a list of strings to parse_dates

filename = '../data/nyc_taxi_2019-01.csv'
df = pd.read_csv(filename, 
                 usecols=['tpep_pickup_datetime',
                          'tpep_dropoff_datetime',
                          'passenger_count', 'total_amount', 'trip_distance'],
                parse_dates=['tpep_pickup_datetime',
                            'tpep_dropoff_datetime'])


In [6]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,total_amount
0,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,9.95
1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,16.30
2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,5.80
3,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,7.55
4,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,55.55


In [7]:
df.dtypes

tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int64
trip_distance                   float64
total_amount                    float64
dtype: object

In [8]:
!head ../data/nyc_taxi_2019-01.csv

In [9]:
# if my file contains 02-04-2022, is that April 2nd, or is it Feb 4th?  It depends if
# you are American or from Europe.

help(pd.read_csv)

Help on function read_csv in module pandas.io.parsers.readers:

read_csv(filepath_or_buffer: 'FilePath | ReadCsvBuffer[bytes] | ReadCsvBuffer[str]', sep=<no_default>, delimiter=None, header='infer', names=<no_default>, index_col=None, usecols=None, squeeze=None, prefix=<no_default>, mangle_dupe_cols=True, dtype: 'DtypeArg | None' = None, engine: 'CSVEngine | None' = None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, skipfooter=0, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=None, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, cache_dates=True, iterator=False, chunksize=None, compression: 'CompressionOptions' = 'infer', thousands=None, decimal: 'str' = '.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, encoding_errors: 'str | None' = 'strict', dialect=None, error_bad_li

In [10]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,total_amount
0,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,9.95
1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,16.30
2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,5.80
3,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,7.55
4,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,55.55


In [11]:
# I want to retrieve the year from each of the pickup datetimes
# to do this, I can use the .dt accessor

df['tpep_pickup_datetime'].dt

In [17]:
# question: how many of the entries in the Jan. 2019 records are really from 2019?
df['tpep_pickup_datetime'].dt.year.value_counts()

2019    7667349
2018        366
2009         50
2008         22
2003          2
2088          2
2001          1
Name: tpep_pickup_datetime, dtype: int64

In [19]:
with open('mydata.csv', 'w') as outfile:
    outfile.write('17-02-2022,hello\n')
    outfile.write('18-02-2022,goodbye\n')
    

In [21]:
mydata_df = pd.read_csv('mydata.csv', header=None)
mydata_df

,0,1
0,17-02-2022,hello
1,18-02-2022,goodbye


In [22]:
mydata_df.dtypes

0    object
1    object
dtype: object

In [23]:
mydata_df = pd.read_csv('mydata.csv', header=None, parse_dates=[0])
mydata_df

/usr/local/lib/python3.10/site-packages/pandas/io/parsers/base_parser.py:1070: UserWarning: Parsing '17-02-2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(
/usr/local/lib/python3.10/site-packages/pandas/io/parsers/base_parser.py:1070: UserWarning: Parsing '18-02-2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return tools.to_datetime(


,0,1
0,2022-02-17,hello
1,2022-02-18,goodbye


In [24]:
mydata_df = pd.read_csv('mydata.csv', header=None, parse_dates=[0], dayfirst=True)
mydata_df

,0,1
0,2022-02-17,hello
1,2022-02-18,goodbye


In [25]:
df['tpep_pickup_datetime'].dt.day_of_week.value_counts()

3    1351516
2    1259695
1    1203843
4    1082795
5    1007797
0     904512
6     857634
Name: tpep_pickup_datetime, dtype: int64

In [26]:
# get a percentage, not a number
df['tpep_pickup_datetime'].dt.day_of_week.value_counts(normalize=True)

3    0.176259
2    0.164284
1    0.157000
4    0.141213
5    0.131432
0    0.117963
6    0.111849
Name: tpep_pickup_datetime, dtype: float64

In [27]:
# how long were the taxi rides?
# in order to know that, we'll need a timedelta!

df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,total_amount
0,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,9.95
1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,16.30
2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,5.80
3,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,7.55
4,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,55.55


In [28]:
# datetime - datetime = time delta

df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']

0         0 days 00:06:40
1         0 days 00:19:12
2         0 days 00:04:10
3         0 days 00:03:20
4         0 days 00:01:36
                ...      
7667787   0 days 00:21:03
7667788   0 days 00:01:08
7667789   0 days 00:00:04
7667790   0 days 00:00:27
7667791   0 days 00:01:19
Length: 7667792, dtype: timedelta64[ns]

In [29]:
# create a new column, trip_time, a timedelta containing the trip time
df['trip_time'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']

In [30]:
df.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,total_amount,trip_time
0,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,9.95,0 days 00:06:40
1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,16.30,0 days 00:19:12
2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,5.80,0 days 00:04:10
3,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,7.55,0 days 00:03:20
4,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,55.55,0 days 00:01:36


In [31]:
df.dtypes

tpep_pickup_datetime      datetime64[ns]
tpep_dropoff_datetime     datetime64[ns]
passenger_count                    int64
trip_distance                    float64
total_amount                     float64
trip_time                timedelta64[ns]
dtype: object

In [33]:
# now what? 
# what if: I want to find all trips that took longer than 12 hours

# you can compare a timedelta in pandas with an int + string (so long as it's a normal measure)
df['trip_time'] > '12 hours'

0          False
1          False
2          False
3          False
4          False
           ...  
7667787    False
7667788    False
7667789    False
7667790    False
7667791    False
Name: trip_time, Length: 7667792, dtype: bool

In [35]:
# use a boolean index to get only those matching rows back
df[df['trip_time'] > '12 hours']

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,total_amount,trip_time
112,2018-12-31 17:22:55,2019-01-01 16:57:23,1,22.59,68.30,0 days 23:34:28
165,2019-01-01 00:53:39,2019-01-02 00:51:48,1,2.60,16.44,0 days 23:58:09
824,2019-01-01 00:27:35,2019-01-02 00:03:49,1,8.28,36.67,0 days 23:36:14
1104,2019-01-01 00:45:54,2019-01-02 00:42:33,2,1.71,13.80,0 days 23:56:39
1108,2019-01-01 00:46:37,2019-01-02 00:29:38,2,16.48,49.30,0 days 23:43:01
...,...,...,...,...,...,...
7664614,2019-01-31 22:45:18,2019-02-01 22:00:05,5,8.61,37.87,0 days 23:14:47
7664706,2019-01-31 19:43:35,2019-02-01 18:54:07,1,2.52,11.80,0 days 23:10:32
7665965,2019-01-31 21:02:14,2019-02-01 20:21:07,1,4.38,23.50,0 days 23:18:53
7666072,2019-01-31 22:13:08,2019-02-01 21:36:12,1,2.56,20.76,0 days 23:23:04


In [36]:
# how much did people pay, on average, for 12+ hour taxi rides?

#        row selector                  column selector
df.loc[df['trip_time'] > '12 hours', 'total_amount'].mean()

19.97441213336676

In [37]:
df.loc[df['trip_time'] > '12 hours', 'trip_distance'].mean()

4.059907746302332